In [1]:
from transformers import ViTModel, ViTFeatureExtractor
import torch
import torch.nn.functional as F
from PIL import Image

### Load Model and Feature Extractor

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ViTModel.from_pretrained('google/vit-base-patch16-224').to(device)
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model.eval()

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/home/seono/anaconda3/lib/python3.12/site-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation()
        )
        (output): ViTOutput(
          (d

### Get Embeddings & Cosine Similarity

In [3]:
def get_embedding(image: Image.Image):
    inputs = feature_extractor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.pooler_output[0]  # or mean pooling
        return F.normalize(embedding, dim=0).cpu()

def cosine_sim(e1, e2):
    return F.cosine_similarity(e1.unsqueeze(0), e2.unsqueeze(0)).item()

In [4]:
def similarity_2(image_path_1, image_path_2):
    image1 = Image.open(image_path_1).convert("RGB")
    image2 = Image.open(image_path_2).convert("RGB")
    return  cosine_sim(get_embedding(image1), get_embedding(image2))

In [7]:
def get(image):
    iamge = Image.open(image).convert("RGB")
    return get_embedding(iamge)

In [9]:
get("patch_5.jpg").size()

torch.Size([768])